**About** : This notebook is used to visualize predictions on the train and test data.
  - Works to visualize predictions from the two previous notebooks, but also from a submission file.
  - Specify the `name`, `log_folder` and `sub` parameters according to what you want to plot.

In [ ]:
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import cv2
import json
import glob
import numpy as np
import pandas as pd
import plotly.express as px

from tqdm.notebook import tqdm

sys.path.append("../code/")

In [ ]:
from params import *
from utils.rle import *

from data.dataset import load_image

from utils.metrics import dice_scores_img
from utils.plots import plot_heatmap_preds, plot_contours_preds

### Data

In [ ]:
THRESHOLD = 0.5

In [ ]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train_2_fix.csv")
df_test = pd.read_csv(DATA_PATH + "sample_submission.csv")

### Experiment

In [ ]:
log_folder = "../logs/2021-05-06/1/"  # b1

In [ ]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [ ]:
preds = glob.glob(log_folder + "pred_*.npy")

## Train viz

In [ ]:
mask_name = "b2dc8411c" 

In [ ]:
idx = [i for i, path in enumerate(preds) if mask_name in path][0]
probs = np.load(preds[idx]).astype(np.float32)

if len(probs.shape) == 3:
    probs = probs.mean(0)

pred = (probs > THRESHOLD).astype(np.uint8)

In [ ]:
img = load_image(os.path.join(TIFF_PATH_2, mask_name + ".tiff"), full_size=False, reduce_factor=2)

In [ ]:
rle = df_mask[df_mask['id'] == mask_name]['encoding']
mask = enc2mask(rle, (img.shape[1], img.shape[0]))

### Plot

In [ ]:
w = 1000
h = int(w *  mask.shape[0] / mask.shape[1])

In [ ]:
# fig = plot_contours_preds(img, pred, mask, w=1, downsize=4)

# fig.update_layout(
#     autosize=False,
#     width=w,
#     height=h,
# )

# fig.show()

In [ ]:
# fig = plot_heatmap_preds(img, probs, mask, w=1, downsize=4)

# fig.update_layout(
#     autosize=False,
#     width=w,
#     height=h,
# )

# fig.show()

## Test viz

### Load image

In [ ]:
name = 'aa05346ff'
fold = 0

img = load_image(f'{DATA_PATH}/test_4/{name}.tiff', full_size=False)

### Using the output of the test inference notebook

In [ ]:
probs = np.load(log_folder + f'pred_{name}_{fold}.npy').astype(np.float32)
pred = (probs > THRESHOLD).astype(np.uint8)

### Using submissions

In [ ]:
shape = df_info[df_info['image_file'] == name + '.tiff'][['width_pixels', 'height_pixels']].values[0]

In [ ]:
sub = pd.read_csv(OUT_PATH + "submission_combine.csv")
rle = sub[sub['id'] == name]['predicted'].values

sub_mask = enc2mask(rle, shape)

sub_mask = cv2.resize(
    sub_mask,
    tuple(list(shape // 4)),
    interpolation=cv2.INTER_NEAREST,
)

You can plot two submissions to compare them

In [ ]:
# sub = pd.read_csv("YOUR_SUB_NAME.csv")
# rle = sub[sub['id'] == name]['predicted'].values

# sub_mask2 = enc2mask(rle, shape)

# sub_mask2 = cv2.resize(
#     sub_mask2,
#     tuple(list(shape // 4)),
#     interpolation=cv2.INTER_NEAREST,
# )

### Plot

In [ ]:
w = 1000
h = int(w * pred.shape[0] / pred.shape[1])

In [ ]:
fig = plot_contours_preds(img, sub_mask, w=2, downsize=2)  # 1 sub
# fig = plot_contours_preds(img, sub_mask, mask=sub_mask2, w=2, downsize=2)  # 2 subs

fig.update_layout(
    autosize=False,
    width=w,
    height=h,
)

fig.show()